Code to create synthetic data based on different distributions.

** Work in progress **

In [15]:
# Import libraries
import pandas as pd
import numpy as np

In [ ]:
# Load the data
data = pd.read_csv('NumericalData.csv')

data = data.drop(data.columns[0], axis = 1)

data.head()

In [ ]:
# Create a dataframe with numerical columns only
nums = data.drop(['subject_id','hadm_id','charttime'], axis = 1)

nums.head()

In [ ]:
# Determine the null values in each column
nums['GCS Total'].replace(0, np.nan, inplace= True)

nan_counts = nums.isnull().sum()

total_counts = len(nums)

nan_percent = (nan_counts/total_counts) * 100

nan_percent

In [ ]:
nums.describe()

In [ ]:
# Calculate mean and variance for each column
smean = nums.mean()

svar = nums.var()

for column in nums.columns:
    print(f"Column: {column}")
    print(f"Mean: {smean[column]}")
    print(f"Variance: {svar[column]}\n")

In [ ]:
# Synthetic data generation
samples = len(nums)

synth_data = pd.DataFrame()

for column in nums.columns:
    mean = smean[column]
    variance = svar[column]
    std_dev = np.sqrt(variance)

    #Sample from a normal distribution
    synthetic_data = np.random.normal(loc = mean, scale = std_dev, size = samples)

    synth_data[column] = synthetic_data   

synth_data

In [ ]:
synth_data.describe()

In [ ]:
# Mean and variance of each columm in the synthetic_data
smean = synth_data.mean()

svar = synth_data.var()

for column in nums.columns:
    print(f"Column: {column}")
    print(f"Mean: {smean[column]}")
    print(f"Variance: {svar[column]}\n")